In [11]:
import csv
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option
import tomotopy as tp
import pandas as pd

origin_path = 'C:/Users/geun/KoreanNewsChronicle/Data/201906.csv'
result_path = "C:/Users/geun/KoreanNewsChronicle/PreProcess/201906_topic.csv"

origin=pd.read_csv(origin_path,encoding='utf-8')

text=origin['text']
print(text)

with open('C:/Users/geun/KoreanNewsChronicle/PreProcess/datas.txt','w',encoding='utf-8') as file:
    for item in text:
        file.write("%s\n" % item)

class Reader:
    def __init__(self, filePath):
        self.file = open(filePath,'r',encoding='UTF-8')
    
    def read(self, id):
        if id == 0: self.file.seek(0)
        return self.file.readline()

reader = Reader('datas.txt')

file = open("한국어불용어100.txt", 'r', encoding="utf-8")
stopword=[]

while True:
    line = file.readline()
    if not line: break
    wordlist = line.split('\t')
    if (wordlist[1].startswith('N')):
        stopword.append(wordlist[0])

print(stopword)
filename="datas.txt"  #파일 경로
file = open(filename, 'r', encoding="UTF-8")
kiwi=Kiwi()
kiwi.extract_add_words(reader.read)
kiwi.prepare()
stopwords = set(stopword)


def tokenize(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('N') and word not in stopwords] #불용어사전 적용


#LDA 모델을 적용해서 토픽 추출, k는 topic 개수
#alpha는 문헌-토픽 디리클레 분포의 하이퍼 파라미터
#eta는 토픽-단어 디리클레 분포의 하이퍼 파라미터 두개다 상수인듯하다
#min_cf는 단어의 최소 장서 빈도수, 전체 문헌내 출현빈도
#min_df는 단어의 최서 문헌 빈도수, 출현한 문헌 숫자수 의미
#tw는 용어 가중치 기법으로, ONE, IDF, PMI를 사용가능, ONE 보다는 PMI나 IDF 둘중 하나 사용 

file = open(result_path, "w", encoding='utf-8', newline = '')
writer = csv.writer(file)
writer.writerow(['index', 'topics'])
for i, line in enumerate(open(filename, encoding='utf-8')): #해당 경로의 파일을 받아와 한 라인씩 model에 추가
    model = tp.LDAModel(k=1, alpha=0.1, eta=0.001, min_cf=3,min_df=1, tw=tp.TermWeight.PMI)
    model.add_doc(tokenize(line)) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    s = ""
    for j in range(0,100):
        model.train(20) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        s = ', '.join(w for w, p in res)
    writer.writerow([i, s])
    # print(i, s)
file.close()
print("End")

topic = pd.read_csv(result_path, encoding='utf-8')

# print(csv)
# print(topic['topics'])

merge = pd.concat([origin, topic['topics']], axis=1)
print(merge)
merge.to_csv('C:/Users/geun/KoreanNewsChronicle/Topic_Abstract/Data/201906.csv', index=False, header=True, encoding="utf-8-sig")
print('merge topics')

0        이번 주부터는 국가에서 부과하는 세금 중에서 우리 생활과 밀접한 관계가 있는 양도소...
1        조선업 생존 위한 불가피한 선택 / 노조 “주총 원천무효” 소송 방침 / 정부, 조...
2        헝가리 유람선 ‘허블레아니’(헝가리어로 인어) 침몰 사고가 발생한 지 사흘째인 31...
3        주거침입 강간미수 혐의…판사 “도망 염려 등 구속 사유 인정” 귀가하는 여성을 몰래...
4        한국과 미국, 일본의 북핵 협상 수석대표들이 하노이 북-미 정상회담 결렬 이후 난관...
                               ...                        
25635    By Robert R. Monroe (former director of the De...
25636    일본이 강제징용 판결에 대해 경제보복 카드를 꺼내든다고 한다. 일본 정부가 TV·스...
25637    국회가 지난 28일 자유한국당을 포함한 여야 5당이 모두 참석한 가운데 본회의를 열...
25638    분단·냉전 상징서 역사적 만남 / 트럼프 “북·미 실무협상 할 것” / 北 비핵화 ...
25639    조국 청와대 민정수석이 30일 남북미 세 정상의 판문점 회동에 대해 "세 지도자의 ...
Name: text, Length: 25640, dtype: object
['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']
End
             date press category  \
0      2019/06/01  국민일보       경제   
1      2019